In [1]:
import pandas as pd
from collections import Counter
from config_datasets import config_datasets
from cargar_dataset import cargar_dataset, graficar_distribucion_clases
from datetime import datetime
from pathlib import Path
# Crear carpeta de resultados si no existe
Path("resultados").mkdir(exist_ok=True)
Path("figuras").mkdir(exist_ok=True)

# Nombre de archivo con fecha y hora
timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
nombre_archivo = f"resultados/reporte_distribucion_{timestamp}.txt"

# Lista de líneas para guardar en el archivo
lineas_resultado = []

for nombre, cfg in config_datasets.items():
    lineas_resultado.append(f"\n🔍 Analizando dataset: {nombre.upper()}")
    print(f"\n🔍 Analizando dataset: {nombre.upper()}")
    try:
        # Cargar el dataset (multiclase)
        X, y, clases = cargar_dataset(
            path=cfg["path"],
            clase_minoria=cfg.get("clase_minoria"),
            col_features=cfg.get("col_features"),
            col_target=cfg.get("col_target"),
            sep=cfg.get("sep", ','),
            header=cfg.get("header"),
            binarizar=cfg.get("binarizar", False),
            tipo=cfg.get("tipo", "tabular")
        )


        # Contar clases originales
        conteo = pd.Series(y).value_counts()
        clase_min_real = conteo.idxmin()
        total = conteo.sum()
        proporcion = (conteo / total * 100).round(2)

        # Mostrar
        print("🎯 Valores únicos del target:", list(conteo.index))
        print("📊 Distribución de clases:")
        # Guardar resultados
        lineas_resultado.append(f"🎯 Valores únicos del target: {list(conteo.index)}")
        lineas_resultado.append("📊 Distribución de clases:")

        for clase, count in conteo.items():
            print(f"   - {clase}: {count} ({proporcion[clase]}%)")
            lineas_resultado.append(f"   - {clase}: {count} ({proporcion[clase]}%)")

        print(f"✅ Clase minoritaria real: {clase_min_real}")
        print(f"⚠️ Clase configurada como minoritaria: {cfg['clase_minoria']}")
        lineas_resultado.append(f"✅ Clase minoritaria real: {clase_min_real}")
        lineas_resultado.append(f"⚠️ Clase configurada como minoritaria: {cfg.get('clase_minoria')}")

        if "clase_minoria" in cfg and cfg["clase_minoria"] is not None:
            if clase_min_real != cfg["clase_minoria"]:
                print("🚨 POSIBLE ERROR DE CONFIGURACIÓN ❗")
                lineas_resultado.append("🚨 POSIBLE ERROR DE CONFIGURACIÓN ❗")
        else:
            print("ℹ️ No se definió clase minoritaria (modo multiclase o imagen).")
            lineas_resultado.append("ℹ️ No se definió clase minoritaria (modo multiclase o imagen).")

        # 🔽 Agregar gráfico descriptivo por dataset
        nombre_figura = f"figuras/{nombre.lower()}_distribucion_{timestamp}.png"
        graficar_distribucion_clases(y, nombre_dataset=nombre, guardar_en=nombre_figura)

    except Exception as e:
        print(f"❌ Error al analizar {nombre}: {e}")
        lineas_resultado.append(f"❌ Error al analizar {nombre}: {e}")

# Guardar a archivo
with open(nombre_archivo, "w", encoding="utf-8") as f:
    f.write("\n".join(lineas_resultado))

print(f"\n📁 Análisis guardado en: {nombre_archivo}")



🔍 Analizando dataset: SHUTTLE
🎯 Valores únicos del target: [1, 4, 5, 3, 2, 7, 6]
📊 Distribución de clases:
   - 1: 45586 (78.6%)
   - 4: 8903 (15.35%)
   - 5: 3267 (5.63%)
   - 3: 171 (0.29%)
   - 2: 50 (0.09%)
   - 7: 13 (0.02%)
   - 6: 10 (0.02%)
✅ Clase minoritaria real: 6
⚠️ Clase configurada como minoritaria: 7
🚨 POSIBLE ERROR DE CONFIGURACIÓN ❗

🔍 Analizando dataset: WDBC
🎯 Valores únicos del target: ['B', 'M']
📊 Distribución de clases:
   - B: 357 (62.74%)
   - M: 212 (37.26%)
✅ Clase minoritaria real: M
⚠️ Clase configurada como minoritaria: M

🔍 Analizando dataset: GLASS
🎯 Valores únicos del target: [2, 1, 7, 3, 5, 6]
📊 Distribución de clases:
   - 2: 76 (35.51%)
   - 1: 70 (32.71%)
   - 7: 29 (13.55%)
   - 3: 17 (7.94%)
   - 5: 13 (6.07%)
   - 6: 9 (4.21%)
✅ Clase minoritaria real: 6
⚠️ Clase configurada como minoritaria: 6

🔍 Analizando dataset: HEART
🎯 Valores únicos del target: [0, 1, 2, 3, 4]
📊 Distribución de clases:
   - 0: 164 (54.13%)
   - 1: 55 (18.15%)
   - 2: 36 (